Getting started with Spacy<br>
Import data.<br>
We repeat the preprocessing from the previous homework.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

In [ ]:
os.chdir( "/content/gdrive/MyDrive/flair" ) 

In [ ]:
path_to_data = os.getcwd() + '/Entity Recognition in Resumes.json'

In [ ]:
myfile = open( path_to_data, "r", encoding = "utf-8" )

imported_data = []

for datum in myfile:
    
    # TODO process data

myfile.close()

# TODO print first line

# TODO print how many resumees were read in

In [ ]:
import json

In [ ]:
mapped_data = [ json.loads( datum ) for datum in imported_data  ]

In [ ]:
## data conversion method
def convert_data(data):
    """
    Creates NER training data in Spacy format from JSON dataset
    Outputs the Spacy training data which can be used for Spacy training.
    """
    text = data['content']
    entities = []
    if data['annotation'] is not None:
        for annotation in data['annotation']:
            # only a single point in text annotation.
            point = annotation['points'][0]
            labels = annotation['label']
            # handle both list of labels or a single label.
            if not isinstance(labels, list):
                labels = [labels]
            for label in labels:
                # dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                entities.append((point['start'], point['end'] + 1, label))
    return (text, {"entities": entities})
   
## TODO using a loop or list comprehension, convert each resume in mapped_data using the convert function above, 
## storing the result
converted_resumes = None
## TODO print the number of resumes in converted resumes 
print( None )

In [ ]:
# TODO filter out the resumees whose entities have no entries.
converted_complete_resumees = None

Up until now, you could reuse the code from the previous notebook, now, something new comes.

In [ ]:
import spacy
nlp = spacy.load('en')
print(nlp)

__nlp__ is Spacy's English language model. For this model, a pretrained NER-model exists.

In [ ]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

__Question 1__: Explain the labels __GPE__, __FAC__, __NORP__.<br>
Which of these labels from __ner__ do you think will Spacy recognize in the resumees?<br>
__Task 1__: choose a resumee.

In [ ]:
# TODO get a single resume text and print it out.
restxt = None
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Next, we let __nlp__ process that single resumee.<br>
__Task 2__: print the results in __doc__. For each result, print the underlying text and the label.

In [ ]:
doc = nlp(restxt)
# TODO  Print the results in doc. For each result, print the text and the label.

__Question 2__: How well did Spacy perform at recognizing the labels for this text?<br>
When Spacy predicted the labels for this resumee, a pretrained model was used.<br>
__Task 3__: print for this resumee the original labels and their corresponding text.

In [ ]:
# TODO print for that resumee the original labels and their corresponding text.

labeled_ents = None

for ent in labeled_ents:


__Question 3__: Compare the performance of the pretrained model __nlp__ and the true labels. Did Spacy perform well? If not, try to explain why.<br>
__Task 4__: Remember last homework? You chose three labels. Select all the resumees, in which all three labels appear.

In [ ]:
# TODO fill in your chosen labels in chosen_entity_labels
chosen_entity_labels = [ None, None, None ]

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates

training_data = gather_candidates( converted_complete_resumees, chosen_entity_labels )
print("Gathered {} training examples".format(len(training_data)))

__Task 5__: Next, we want to remove all other entities, since we only want to train NER for the three entities in __chosen_entity_labels__.

In [ ]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable. X shall contain all
## the resumees from training_data, but their entity annotations shall be filtered using the function from above.
X = None

__Task 6__: Some resumees cause trouble. We filter these out with the following lines of code.<br>
First, use __add_label__ to add your chosen labels to the __ner__ model.

In [ ]:
from spacy.gold import GoldParse 

# TODO add your labels 
ner.add_label( None )

ner.add_label( None )

ner.add_label( None )

nlp.begin_training()

good = []

for item in X:
  
  text = nlp.make_doc( item[ 0 ] )

  try:
    
    gold = GoldParse( text, entities = item[ 1 ][ "entities" ] )

  except:

    continue
  
  try:
    
    nlp.update( [ text ], [ gold ], drop = 0.3 )

  except:

    pass

  else:

    good.append( item )

print( "Number of good samples: " + str( len( good ) ) )

print( "" )

print( "" )

print( "Number of bad sampples: " + str( len( X ) - len( good ) ) )

For a machine learning model, it is essential to be able to generalize. Only a model, that can generalize well is able to process new data in a meaningful way. Therefore, one usually separates the data set into two sets: the training set and the test set. The training set is used to train the model. The test set is used to evaluate the performance of the model on unseen data.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split( [ item[ 0 ] for item in good ], [ item[ 1 ] for item in good ], test_size = 0.3 )

__Task 7__: Complete the following code. Shuffle __new_index__. Create the data sets __x_shuffled__ and __y_shuffled__. Use these to create minibatches, iterate over these minibatches, preprocess the data in a given minibatch using __nlp.make_doc__ and __GoldParse__. Employ __nlp.update__ to update the model using these preprocessed data.

In [ ]:
import numpy as np

nlp.begin_training()

new_index = np.arange( len( x_train ) )

x_data = np.array( x_train )

y_data = np.array( y_train )

for i in range( 20 ):

  # TODO shuffle new_index

  x_shuffled = #TODO create x_shuffled from x_data by using the shuffled new_index

  y_shuffled = #TODO create y_shuffled from y_data by using the shuffled new_index

  # TODO
  # divide the data in x_shuffled and y_shuffled into minibatches of identical size
  # iterate over these minibatches
  # preprocess the data in a minibatch using nlp.make_doc and GoldParse
  # use these preprocessed data and nlp.update to train the model

__Question 4__: Why did we shuffle the data?<br> 
Why did we employ mini batches?<br>
Reasearch the term __epoch__ in machine learning. How many epochs of training do we employ?<br>
__Task 8__: Next, we choose one resumee and print it.

In [ ]:
resume = None

print( resume )

__Task 9__: we process this resumee using __nlp__. Print for all items in __doc.ents__ the predicted label and the corresponding text. Then print the correct labels and their corresponding text for that resumee with data from __y_test__.

In [ ]:
doc = nlp( resume )

# TODO
# print for all the items in doc.ents the predicted label and the corresponding text

for item in doc.ents:

# TODO
# print the correct labels and their corresponding text for that resumee with data from y_test



__Question 5__: What labels did the model predict correctly?<br> 
Where appeared problems?<br> 
How can you explain the problems?<br>
__Question 6__: We can evaluate the performance of the model using 4 metrics: the __Accuracy__, the __Precision__, the __Recall__ and __F1__.<br>
Inform yourself on these metrics. How are they defined? Explain the concept of __True Positive__, __True Negative__, __False Positive__ and __False Negative__. Use these to define  the __Accuracy__, the __Precision__, the __Recall__ and __F1__, and also give the formula for each of these.<br>
__Task 10__: Complete the following code. Call __make_bilou_df__ with a resume from the test set and store result in __bilou_df__ variable.

In [ ]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc, [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df( nlp, None )
display(bilou_df)  

Inform yourself on the [BILUO](https://spacy.io/usage/linguistic-features#accessing-ner) scheme.<br>
__Question 7__: Why do you think is it better to tag entities using this scheme (consider names of humans, descriptions of items in a shop)?<br>
__Task 11__: employ pandas dataframe api to get a subset where predicted and true labels are the same. Compute the accuracy using the formula you researched above.

In [ ]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = None
## TODO compute the accuracy 
accuracy = None

print("Accuracy on one resume: ",accuracy)

The __accuracy__ is not 100%. Therefore, we want to have a look at those tokens, where the predicted and the true value differ.<br>
__Task 12__: create a dataframe diff_df where the predicted values and the true values differ

In [ ]:
# TODO create a dataframe diff_df where the predicted values and the true values differ
diff_df = None
display(diff_df)

Since we only considered one resumee, we now make this comparison for the whole test set.<br>
__Task 13__: Complete the following code for the computation of the overall accuracy.

In [ ]:
doc_accuracy = []

for i in range( len( x_test ) ):

  resume = None

  bilou_df = make_bilou_df(nlp,resume)

  same_df = None

  doc_accuracy.append( None )

total_acc = np.mean( doc_accuracy )
print("Accuracy: ",total_acc)

So we got an __accuracy__ of about 90% on average. This is quite good considering, that we only considered about 300 cases for training.<br>
__Task 14__: Next, we want to find out, what the model did, when it went wrong. We only consider 5 resumees.<br>
Complete the following code.

In [ ]:
for i in range( 5 ):

  resume = None

  bilou_df = make_bilou_df(nlp,resume)

  difference_df = None

  # TODO: print, where the labels from Spacy and the annotation differ. Print the text, the predicted and the true labels.

__Question 8__: What was predicted, when the prediction differed from the true label?<br>
What do you think is necessary for computing the accuracy on token level?<br> 
What is the advantage of computing the accuracy on token level?<br>
__Task 15__: Complete the following code.

In [ ]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for i in range( len( x_test ) ):
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 

        resume = None
        
        tres_df = make_bilou_df(nlp,resume)

        ## calculate true false positives and false negatives for each resume
        
        tp = None
        
        fp = None
        
        fn = None

        ## aggregate result for each resume to totals
        true_positives = true_positives + None
        false_positives = false_positives + None
        false_negatives = false_negatives + None
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out
    ## also: prevent division by zero without raising errors. Explain your choice
    
    row = [label,precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame( data, columns = [ "Label", "Precision", "Recall", "F1" ] )
display(metric_df)

__Question 9__: Explain from these statistics how well __nlp__ performs.<br>
__Task 16__: Compute for each metric (Precision, Recall, F1) the mean values.

In [ ]:
for label in [ "Precision", "Recall", "F1" ]:
    
    # Compute mean and print

__Question 10__: What do you learn, when you compare the performance of the model on the token level with the performance of the model on the global level from above?<br>
Next, we prepare data for flair.

In [ ]:
train = [ [ x_train[ i ], y_train[ i ] ] for i in range( len( x_train ) ) ]

test = [ [ x_test[ i ], y_test[ i ] ] for i in range( len( x_test ) ) ]

__Task 17__: Complete the following code.

In [ ]:
# prepare data
training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]

test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]


# set up paths
path_to_training_file = os.getcwd() + "/training_data.csv"

path_to_test_file = os.getcwd() + "/test_data.csv"



# make sure, that if the corresponding files exist, they are emptied
if os.path.isfile( path_to_training_file ):

  open( path_to_training_file, "w" ).close()

if os.path.isfile( path_to_test_file ):

  open( path_to_test_file, "w" ).close()


# open empty files
training_file = open( path_to_training_file, "a", encoding = "utf-8" )
    
test_file = open( path_to_test_file, "a", encoding = "utf-8" )


for item in training_data_as_bilou:

  # TODO remove all tokens like "", " ", "\n" by ignoring them
  # for all other tokens do the following:
  # create a string s: s = token + " " + label + "\n"
  # if the label is "-", then write s = token + " O\n"
  #
  # write this newly created string to file
  # if this newly created string contains ".", then also write a 
  # newline to file that only contains "\n"
  #
  # Using this scheme, each line in the resulting files corresponds either to an empty line or a token.
  # Flair assembles a block of nonempty lines into a sentence. Therefore, the empty line
  # is a signal for Flair that the current sentence is finished. Therefore, we extracted
  # the whitespaces above.

for item in test_data_as_bilou:
    
  # TODO the same as above.


training_file.close()

test_file.close()

Start Flair

In [ ]:
pip install flair

In [ ]:
from flair.data import Corpus
from flair.data_fetcher import NLPTaskDataFetcher


# your training file name
data_folder = os.getcwd() 

train_file = "training_data.csv"

# your training file name
test_file = "test_data.csv"

# when we wrote the data files, each row was either empty to signal the end
# of a sentence to Flair, or the line contained a token, a white space and a label.
# In the next line, we assign, that the token is the "text", and that the label is 
# "ner" label
columns =  {0: 'text', 1: 'ner'}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)